# BSS-AUS: Basketball Statistic System (AUS)

This system allows to **incrementally** build a database of _stint_ (advance) statistics for each game and each team.

A **stint** is a lineup of players who play together in different interval periods across the game. The system will build the stints for each team from the play-by-play data and compute various statistics.

The data comes as a raw JSON file:

https://fibalivestats.dcd.shared.geniussports.com/data/2087737/data.json

In [1]:
# Let's first load all required packages...
import os
import pandas as pd

from config import *
import bball_stats
import tools

## 1. Compute stat tables for various games


## 1. Define games to scrape and saved data

First, setup the games we want to scrape and compute, as well as the existing data stored in file to append to.

In [2]:
# games to be computed
# format: (game id, game no for team 1, game no for team 2)
games = [(1999318, 1, 1), (2087737, 2, 2), (2090351, 3, 3)]

reload = True
file_stats_df = os.path.join(DATA_DIR, "stats_df.pkl")
file_games_df = os.path.join(DATA_DIR, "games_df.pkl")

## 2. Compute stat and game tables

Now, let us run the system that scrapes the games' data, compute stats and game info, and adds them to the initial tables of stats and games.

In [14]:
init_stats_df = None
game_df = None
if os.path.exists(file_stats_df) and not reload:
    # load the stat dataframe already stored as a file
    print(f"Loading initial stats df: {file_stats_df}")
    init_stats_df = pd.read_pickle(file_stats_df)
    game_df = pd.read_pickle(file_games_df)
    existing_games = init_stats_df.game_id.unique()
else:
    existing_games = []

stats_dfs = [init_stats_df] if init_stats_df is not None else []
games_data = []
for game in games:
    game_id, game_no1, game_no2 = game
    if game_id in existing_games:
        print(f"Game {game_id} is already in table")
        continue
    print(f"Computing game {game_id}...")

    # now compute the actual stats for the game
    df, team1, team2 = bball_stats.build_game_stints_stats_df(game_id)
    date_game = tools.get_game_info(game_id)    # extract date of game from HTML
    print(f"\t .... done: {team1[0]} ({team1[1]}) vs {team2[0]} ({team2[1]}) on {date_game}")

    # fill game info
    df.insert(0, 'game_id', game_id)
    df.insert(3, 'game_no1', game_no1)
    df.insert(4, 'game_no2', game_no2)
    stats_dfs.append(df)

    # build game dataframe table
    games_data.append({'game_id': game_id,
                       "date" : date_game['date'],
                       "team1": team1[0], "team2": team2[0],
                       "s1": team1[1], "s2": team2[1],
                       "game_no1": game_no2, "game_no2": game_no2,
                       "winner": 1 if team1[1] > team2[1] else 2,
                        "venue" : date_game["venue"]}
                      )

# put all dfs together into a single dataframe
stats_df = pd.concat(stats_dfs)
stats_df.reset_index(inplace=True, drop=True)
stats_df.sample(5)

if game_df is not None:
    games_df = pd.concat([game_df, pd.DataFrame(games_data)])
    games_df.reset_index(inplace=True, drop=True)
else:
    games_df = pd.DataFrame(games_data)

Computing game 1999318...
====> Game Melbourne United (United) vs New Zealand Breakers (Breakers)
Play-by-play df for game 1999318: (644, 18)
Starters for Melbourne United: {'M. Peatling', 'M. Dellavedova', 'C. Agada', 'C. Goulding', 'J. Lual-Acuil'}
Starters for New Zealand Breakers: {'H. Besson', 'T. Abercrombie', 'P. Siva', 'R. Loe', 'F. Delany'}
	 .... done: Melbourne United (90) vs New Zealand Breakers (84) on {'venue': 'Melbourne Sports & Aquatic Centre', 'date': datetime.datetime(2021, 11, 13, 0, 0)}
Computing game 2087737...
====> Game Melbourne United (United) vs Tasmania JackJumpers (JackJumpers)
Play-by-play df for game 2087737: (566, 18)
Starters for Melbourne United: {'J. Lual-Acuil Jr', 'S. Ili', 'M. Dellavedova', 'C. Agada', 'J. White'}
Starters for Tasmania JackJumpers: {'J. Magette', 'F. Krslovic', 'J. Mcveigh', 'J. Adams', 'M. Kenyon'}
	 .... done: Melbourne United (73) vs Tasmania JackJumpers (76) on {'venue': 'John Cain Arena', 'date': datetime.datetime(2022, 5, 2, 

If we want we can do some sanity checks, before saving to disk:

In [10]:
# Check the two df computed
games_df
# stats_df
# stats_df.loc[4]

,game_id,date,team1,team2,s1,s2,game_no1,game_no2,winner
0,1999318,"{'venue': 'Melbourne Sports & Aquatic Centre',...",Melbourne United,New Zealand Breakers,90,84,1,1,1
1,2087737,"{'venue': 'John Cain Arena', 'date': 2022-05-0...",Melbourne United,Tasmania JackJumpers,73,76,2,2,2
2,2090351,"{'venue': 'MyState Bank Arena', 'date': 2022-0...",Tasmania JackJumpers,Sydney Kings,86,90,3,3,2


In [5]:
# (ortg, drtg) should mirror (drtg_opp, ortg)
stats_df.iloc[41][['poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]]

poss         10.88
ortg         73.53
drtg        101.21
poss_opp      9.88
ortg_opp    101.21
drtg_opp     73.53
Name: 41, dtype: object

## 3. Save stats and games to files

We now save the full dataframes (stats and games) in various formats: binary (pickle), csv, and Excel.

This will allows us to re-load that data later to add more games to it quicker.

In [6]:

import os

stats_df.to_pickle(os.path.join(DATA_DIR, "stats_df.pkl"))
games_df.to_pickle(os.path.join(DATA_DIR, "games_df.pkl"))

stats_df.to_csv(os.path.join(DATA_DIR, "stats_df.csv"))
games_df.to_csv(os.path.join(DATA_DIR, "games_df.csv"))

with pd.ExcelWriter(os.path.join(DATA_DIR, 'stats_df.xlsx')) as writer:
    stats_df.to_excel(writer, sheet_name='STATS')
    games_df.to_excel(writer, sheet_name='GAMES')
games_df.to_excel(os.path.join(DATA_DIR, "games_df.xlsx"))


### 3. Inspection & analysis

We use [dtale](https://pypi.org/project/dtale/) package for this.

In [7]:
import dtale

dtale.show(stats_df)
# dtale.show(stats_df[['tno', 'stint', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]])

## Web page analysis

In [8]:
tools.get_game_info(1976446)
# import datetime
# datetime.datetime.strptime("3/2/89", "%d/%m/%y")

{'venue': 'MyState Bank Arena', 'date': datetime.datetime(2021, 12, 3, 0, 0)}